In [ ]:
import networkx as nx # type: ignore
import random
from shapely.geometry import Point # type: ignore
from tqdm import tqdm # type: ignore
import geopandas as gpd # type: ignore
import numpy as np # type: ignore

%run toy_city_generation.ipynb
%run city_utilities.ipynb
%run real_city_generation.ipynb

%pip install osmnx matplotlib

In [ ]:
def get_valid_area(G, trip, tolerance=0.1):
    start_nodes = {n for n, data in G.nodes(data=True) if data.get("zone") == trip["start_zone_id"]}
    end_nodes = {n for n, data in G.nodes(data=True) if data.get("zone") == trip["end_zone_id"]}

    if not start_nodes or not end_nodes:
        return [], [], 0, 0

    valid_start_points = set()
    valid_end_points = set()

    for s in start_nodes:
        for e in end_nodes:
            try:
                path_length = nx.shortest_path_length(G, s, e, weight="length")
                path_time = nx.shortest_path_length(G, s, e, weight="travel_time")

                length_ratio = abs(path_length - trip["distance"]) / trip["distance"]
                time_ratio = abs(path_time - trip["travel_time"]) / trip["travel_time"]

                if length_ratio < tolerance and time_ratio < tolerance:
                    valid_start_points.add(s)
                    valid_end_points.add(e)

            except nx.NetworkXNoPath:
                continue

    fraction_start = len(valid_start_points) / len(start_nodes) if start_nodes else 0
    fraction_end = len(valid_end_points) / len(end_nodes) if end_nodes else 0

    return list(valid_start_points), list(valid_end_points), fraction_start, fraction_end

In [ ]:
# Main function

USE_FAKE_CITY = False

if USE_FAKE_CITY:
    zones, street_graph, trips, width, height = generate_synthetic_city_data() # type: ignore
    visualise(street_graph, zones, width, height, trips=trips) # type: ignore
else:
    zones, street_graph, trips = generate_real_city_data() # type: ignore
    print("Visualising real city...")
    display_real_city(street_graph, zones, trips) # type: ignore

In [ ]:
print("Getting valid area")
# Get the average fraction of valid start and end points
start_fractions = []
end_fractions = []
for trip in trips:
    print(trip)
    valid_start_points, valid_end_points, fraction_start, fraction_end = get_valid_area(street_graph, trip)
    start_fractions.append(fraction_start)
    end_fractions.append(fraction_end)

print("Average fraction of valid start points:", np.mean(start_fractions))
print("Average fraction of valid end points:", np.mean(end_fractions))